In [0]:
dbutils.secrets.list(scope="fomular1-scope")

fomular1_key = dbutils.secrets.get(scope="fomular1-scope", key="formuladl1-key-account")

spark.conf.set("fs.azure.account.key.fomular1dl1.dfs.core.windows.net", fomular1_key)

In [0]:
df = spark.read.parquet("abfss://presentation@fomular1dl1.dfs.core.windows.net/race_results")
display(df)

In [0]:

from pyspark.sql.functions import sum, desc, asc, col, when, any_value, count

driver_standing_df = df.groupBy("race_year","driver_nationality","team","driver_name").agg(sum("points").alias("total_points"), count(when(col("position") == 1, True)).alias("wins"))

In [0]:
display(driver_standing_df.filter("race_year = 2020"))

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import rank

driver_rank_spec = Window.partitionBy("race_year").orderBy(desc("total_points"), desc("wins"))
driver_ranked_df = driver_standing_df.withColumn("rank", rank().over(driver_rank_spec))

In [0]:
display(driver_ranked_df.filter("race_year = 2020"))

In [0]:
driver_ranked_df.write.mode("overwrite").format("parquet").saveAsTable("f1_presentation.driver_standing")